<a href="https://colab.research.google.com/github/atlimited/llm_train/blob/main/deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -r deepseek
!cp -r drive/MyDrive/deepseek .

In [3]:
!cp drive/MyDrive/train.txt .

In [4]:
!git clone https://github.com/huggingface/transformers.git -b v4.38.2

fatal: destination path 'transformers' already exists and is not an empty directory.


In [5]:
!cp transformers/examples/pytorch/language-modeling/run_clm.py .

In [6]:
!ls

deepseek  drive  output  run_clm.py  sample_data  train.txt  transformers


In [7]:
!pip install -r transformers/examples/pytorch/language-modeling/requirements.txt

In [8]:
!pip install flash-attn --no-build-isolation

In [9]:
%%writefile /content/deepseek/config.json

{
  "architectures": [
    "DeepseekForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_deepseek.DeepseekConfig",
    "AutoModel": "modeling_deepseek.DeepseekModel",
    "AutoModelForCausalLM": "modeling_deepseek.DeepseekForCausalLM"
  },
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "first_k_dense_replace": 1,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 10944,
  "max_position_embeddings": 4096,
  "model_type": "deepseek",
  "moe_intermediate_size": 1408,
  "moe_layer_freq": 1,
  "n_routed_experts": 8,
  "n_shared_experts": 2,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts_per_tok": 6,
  "num_hidden_layers": 14,
  "num_key_value_heads": 16,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "scoring_func": "softmax",
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.0",
  "use_cache": true,
  "vocab_size": 102400
}

Overwriting /content/deepseek/config.json


In [10]:
import json
from deepseek.modeling_deepseek import DeepseekForCausalLM
from deepseek.configuration_deepseek import DeepseekConfig

model_dir = 'deepseek'
config_file = './deepseek/config.json'
with open(config_file, 'r') as f:
    config = json.load(f)
    config = DeepseekConfig.from_dict(config)

model = DeepseekForCausalLM(config)

model.save_pretrained(model_dir)

In [11]:
!python run_clm.py \
    --model_name_or_path=./deepseek \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=30 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --overwrite_output_dir=true \
    --use_fast_tokenizer=False \
    --trust_remote_code=true

2024-04-22 04:15:03.443065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 04:15:03.443113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 04:15:03.444806: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 04:15:04.677193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/22/2024 04:15:07 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/22/2024 04:15:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
ac